sample source : https://chrisalbon.com/machine_learning/trees_and_forests/feature_selection_using_random_forest/

### Feature Selection using Random Forest Classifier

In [61]:
#import packages 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets as ds

C:\Users\henci\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
#define required functions

In [143]:
def create_and_train_random_forest_model(train_ds, test_ds, predictors, target):
    #n_estimator - no of trees 
    #random_state - random no generation for sample
    #n_jobs -1 - use all jobs in parallel 
    model_rf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
    
    #train the model 
    model_rf.fit(train_ds[predictors].values, train_ds[target].values)
    
    #print the model coefficient values 
    print('********Random Forest Model Craeted***************')
    print('********Predictor Name and Coefficient Weight*******')
    for feature in zip(predictors, model_rf.feature_importances_):
        print(feature)

    return model_rf


In [202]:
def select_features(model, train_ds, predictors, target, threshold):
    select_model = SelectFromModel(model, threshold=threshold)
    select_model.fit(train_ds[predictors].values, train_ds[target].values)
    
    #print the selected features 
    print('Selected Feature Names')
    selected_features=[]
    for selected_feature_index in select_model.get_support(indices=True):
        print(predictors[selected_feature_index])
        selected_features.append(predictors[selected_feature_index])
        
    return selected_features

In [160]:
def compute_and_print_accuracy(model, test_ds, predictors, target):
    predict_result = model.predict(test_ds[predictors].values)
    auc_score = accuracy_score(test_ds[target].values, predict_result)
    
    print('Accuracy Score : ', auc_score)
    return auc_score

In [17]:
#load dataset
iris_ds = ds.load_iris()
X = iris_ds.data
Y = iris_ds.target


In [134]:
feature_all = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width']
target = ['class']
features_and_target = feature_all.copy()
features_and_target.append(target)

print('features : ', feature_all)
print('target : ', target)
print('features and target: ', features_and_target)

features :  ['sepal-length', 'sepal-width', 'petal-length', 'petal-width']
target :  ['class']
features and target:  ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', ['class']]


In [135]:
#split train and test dataset 
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=0)

In [136]:
#prepare combined train and test dataset 

#prepare train data set
train_ds = np.zeros( shape = (train_x.shape[0] , (train_x.shape[1] + 1)))
train_ds[:, 0:-1] = train_x
train_ds[:,-1] = train_y

#prepare test data set 
test_ds = np.zeros(shape=(test_x.shape[0], test_x.shape[1]+1))
test_ds[:,0:-1] = test_x
test_ds[:, -1] = test_y

In [166]:
#prepare data frame
train_ds_df = pd.DataFrame(train_ds, columns=[features_and_target])
test_ds_df = pd.DataFrame(test_ds, columns=[features_and_target])

TypeError: unhashable type: 'list'

#### 1. Train the Rondom Forest Classifer with all features and print the coefficients and accuracy

In [145]:
#create and train random forest model
model_rf = create_and_train_random_forest_model(train_ds_df, test_ds, feature_all, target)


C:\Users\henci\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


********Random Forest Model Craeted***************
********Predictor Name and Coefficient Weight*******
('sepal-length', 0.1073721771400413)
('sepal-width', 0.025838001950909274)
('petal-length', 0.4581071734188299)
('petal-width', 0.40868264749021965)


In [161]:
#compute and print the model accuracy
compute_and_print_accuracy(model_rf, test_ds_df, feature_all, target)




Accuracy Score :  0.9777777777777777


0.9777777777777777

### 2. Train the Random Forest Classifier with selected features and print the coefficients and accuracy. 

In [203]:
selected_features = select_features(model_rf, train_ds_df,feature_all,target, None)

C:\Users\henci\anaconda3\lib\site-packages\sklearn\feature_selection\_from_model.py:222: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


Selected Feature Names
petal-length
petal-width


In [206]:
#create and train model with selected features
model_rf_customized = create_and_train_random_forest_model(train_ds_df, test_ds_df, selected_features, target)

C:\Users\henci\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


********Random Forest Model Craeted***************
********Predictor Name and Coefficient Weight*******
('petal-length', 0.5012936278844757)
('petal-width', 0.49870637211552415)


In [210]:
#compute and print model accuracy
compute_and_print_accuracy(model_rf_customized, test_ds_df, selected_features, target)

Accuracy Score :  0.9777777777777777


0.9777777777777777

In my samples, i have got the same accuracy in both the models. But in the sample, selected features give lower accuracy.